In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')

import plotly.offline as py 
py.init_notebook_mode(connected=True) # this code, allow us to work with offline plotly version
import plotly.graph_objs as go # it's like "plt" of matplot
#import plotly.tools as tls # It's useful to we get some tools of plotly

In [ ]:
import folium
from folium import plugins
from folium import Map
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster

from folium.plugins import MarkerCluster

In [ ]:
df = pd.read_csv("bases/questao1_coronavirus.csv", delimiter=";")

Usaremos como base de informação o site 
https://brasil.io/dataset/covid19/boletim/
que tem a siguinte documentação:

date: data de coleta dos dados no formato YYYY-MM-DD.

state: sigla da unidade federativa, exemplo: SP.

city: nome do município (pode estar em branco quando o registro é referente ao estado, pode ser preenchido com Importados/Indefinidos também).

new_confirmed: número de novos casos confirmados desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).

new_deaths: número de novos óbitos desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).

estimated_population_2019: população estimada para esse município/estado em 2019, segundo o IBGE (acesse o script que faz o download e conversão dos dados de população).

In [5]:
df.sample(10)

,date,state,city,new_confirmed,new_deaths,estimated_population_2019
187308,2020-05-06,RS,Victor Graeff,0,0,2882.0
137289,2020-05-22,SP,Jaboticabal,0,0,77263.0
141407,2020-05-20,AL,Paripueira,0,0,13176.0
148486,2020-05-18,AL,Jacuípe,0,0,7021.0
230606,2020-04-10,MG,Boa Esperança,0,0,40127.0
41263,2020-06-13,PI,Corrente,0,0,26644.0
173821,2020-05-11,SP,Ibirá,0,0,12393.0
217152,2020-04-21,MG,Betim,0,0,439340.0
189833,2020-05-05,RS,Serafina Corrêa,9,0,17502.0
10197,2020-06-19,BA,Ibirataia,2,0,15312.0


In [6]:
def estado_especifico(valor):
    estados = df.drop("city", 1)
    df_estado = estados.loc[estados.state==valor, :]
    df_estado = df_estado.groupby('date').sum()
    df_estado[['new_confirmed','new_deaths']] = df_estado[['new_confirmed','new_deaths']].cumsum()
    return df_estado

In [7]:
cidades = pd.read_csv("bases/cidades_brasil.csv")
cidades = cidades.rename(columns={"nome":"city"})

In [8]:
cidades.head()

,codigo_ibge,city,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiânia,-16.19700,-48.7057,0,52
3,3100203,Abaeté,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


In [9]:
result = pd.merge(df, cidades, on='city')

In [10]:
result = result.dropna()
result['coord'] = result[['latitude','longitude']].apply(tuple, axis=1)

In [11]:
result.head()

,date,state,city,new_confirmed,new_deaths,estimated_population_2019,codigo_ibge,latitude,longitude,capital,codigo_uf,coord
0,2020-06-21,AC,Acrelândia,0,0,15256.0,1200013,-9.82581,-66.8972,0,12,"(-9.82581, -66.8972)"
1,2020-06-20,AC,Acrelândia,0,0,15256.0,1200013,-9.82581,-66.8972,0,12,"(-9.82581, -66.8972)"
2,2020-06-19,AC,Acrelândia,0,0,15256.0,1200013,-9.82581,-66.8972,0,12,"(-9.82581, -66.8972)"
3,2020-06-18,AC,Acrelândia,1,0,15256.0,1200013,-9.82581,-66.8972,0,12,"(-9.82581, -66.8972)"
4,2020-06-17,AC,Acrelândia,2,0,15256.0,1200013,-9.82581,-66.8972,0,12,"(-9.82581, -66.8972)"


In [15]:
result.shape

(268320, 12)

In [12]:
basemap = folium.Map(
                    width="100%",
                    height="100%",
                    location= [-15.788497, -47.879873],
                    zoom_start=4
)

In [58]:
for i in range(0, 3):
    coordenada = result.iloc[i]['coord']
    print(result['coord'][i:i+1])

0    (-9.82581, -66.8972)
Name: coord, dtype: object
1    (-9.82581, -66.8972)
Name: coord, dtype: object
2    (-9.82581, -66.8972)
Name: coord, dtype: object


In [ ]:
for i in range(0, len(result)):
    for x in range(0, int(result.iloc[i]['new_confirmed'])):
        basemap.add_child(FastMarkerCluster(result['coord'][i:i+1]))

In [50]:
for i in range(0, len(result)):
    coordenada = pd.Series(result.iloc[i]['coord'])
    for x in range(0, int(result.iloc[i]['new_confirmed'])):
        basemap.add_child(FastMarkerCluster(coordenada))

ValueError: Expected two (lat, lon) values for location, instead got: (-9.82581,).

In [55]:
result['coord'][0:1]

0    (-9.82581, -66.8972)
Name: coord, dtype: object

In [56]:
type(result['coord'][0:1])

pandas.core.series.Series

In [17]:
basemap.add_child(FastMarkerCluster(result['coord']))

TypeError: 'float' object is not subscriptable

In [13]:
marker_cluster = MarkerCluster().add_to(basemap)

In [ ]:
for i in range(0, len(result)):
    folium.Marker(location = [result.iloc[i]['latitude']], 
                  popup=result['city'][result.iloc[i]['latitude'], result.iloc[i]['longitude']]
                 ).add_to(marker_cluster)


In [ ]:
basemap = basemap.add_child(plugins.HeatMap(coordenada))

In [ ]:
basemap

In [ ]:
for i in range(0, len(result)):
    folium.Circle(
        location= [result.iloc[i]['latitude'], result.iloc[i]['longitude']],
        color = '#00FF69',
        fill = '#00A1B3',
        tooltip = '<li><bold> Cidade: ' + str(result.iloc[i]['city']) +
                  '<li><bold> Estado: ' + str(result.iloc[i]['state']) +
                  '<li><bold> Casos : ' + str(result.iloc[i]['new_confirmed']) +
                  '<li><bold> Obitos: ' + str(result.iloc[i]['new_deaths']),
        radius = (result.iloc[i]['new_confirmed']**1.1)
    ).add_to(basemap)

In [ ]:
estado_sp = estado_especifico("SP")

In [ ]:
estado_ce = estado_especifico("CE")

In [ ]:
trace = [go.Scatter(x = estado_ce.index, y = estado_ce.new_confirmed),
        go.Scatter(x = estado_sp.index, y = estado_sp.new_confirmed)]

layout = dict(
    title='Ocorrências de Roubo de Celular',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='Último Mês',
                     step='month',
                     stepmode='backward'),
                dict(label='Tudo',
                     step='all'
                    )
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = dict(data=trace, layout=layout)
py.iplot(fig)

In [ ]:
dist_df = df_estado.reset_index(level=[0,1])
dist_df[dist_df["state"]=="CE"].head()

In [ ]:
dist_df = df_estado.reset_index(level=[0,1])

In [ ]:
dist_df.head()

In [ ]:
lista_estado = dataframe_questao_1.state.unique()
populacao = 0
for i in lista_estado:
    print(df_estado[i].estimated_population_2019.max())

In [ ]:
def estado(valor):
    estados = dataframe_questao_1.drop("city", 1)
    df_estado = estados.loc[estados.state==valor, :]
    df_estado = df_estado.groupby('date').sum()
    df_estado[['new_confirmed','new_deaths']] = df_estado[['new_confirmed','new_deaths']].cumsum()
    return df_estado

In [ ]:
teste = estado("SP")
teste.head(10)

In [ ]:
teste =teste['2020-02-27' :'2020-03-02']

In [ ]:
teste.head(10)